In [ ]:
%env RERUN_NOTEBOOK_ASSET=serve-local

In [ ]:
from __future__ import annotations

import ipywidgets as widgets
from IPython.display import display

# Create the text input widget
text_input = widgets.Text(
    value='',
    placeholder='Type something here',
    description='Input:',
    disabled=False
)

# Create the button widget
button = widgets.Button(
    description='Print Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to print input',
    icon='check'
)

# Create an output widget to display the results
output = widgets.Output()

# Define the button click event handler
def on_button_clicked(b):
    with output:
        # Clear previous output
        output.clear_output()
        # Print the current value of the text input
        print(f"You entered: {text_input.value}")

# Connect the button click event to the handler function
button.on_click(on_button_clicked)


from rerun_notebook import Viewer, ViewerCallbacks

viewer = Viewer(
    url="https://app.rerun.io/version/nightly/examples/raw_mesh.rrd",
    panel_states={
        "blueprint": "hidden",
        "selection": "expanded",
        "time": "collapsed",
    },
)

class AnnotationCallbackHandler(ViewerCallbacks):
  def on_selection_change(self, selection):
    pass

viewer.register_callbacks(AnnotationCallbackHandler())

# Display all widgets
display(viewer)
display(text_input)
display(button)
display(output)
